In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import dipy.reconst.sfm as sfm
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.core.gradients as grad
import dipy.reconst.cross_validation as xval
import dipy.data as dpd

In [3]:
import utils
import model as mm 
from model import Model, BiExponentialIsotropicModel

In [4]:
mm.my_responses

[[0.001, 0.0, 0.0],
 [0.001, 0.00050000000000000001, 0.00050000000000000001],
 [0.0015, 0.0, 0.0],
 [0.0015, 0.00050000000000000001, 0.00050000000000000001],
 [0.0015, 0.001, 0.001]]

In [5]:
data = utils.read_data()

In [6]:
data1 = data['seen']['signal'][:, 0][::2]
bvals1 = data['seen']['bvals'][::2]
bvecs1 = data['seen']['bvecs'][::2]
delta1 = data['seen']['delta'][::2]
Delta1 = data['seen']['Delta'][::2]
te1 = data['seen']['TE'][::2]
g1 = data['seen']['g'][::2]

data2 = data['seen']['signal'][:, 0][1::2]
bvals2 = data['seen']['bvals'][1::2]
bvecs2 = data['seen']['bvecs'][1::2]
delta2 = data['seen']['delta'][1::2]
Delta2 = data['seen']['Delta'][1::2]
te2 = data['seen']['TE'][1::2]
g2 = data['seen']['g'][1::2]

In [7]:
gtab1 = grad.gradient_table(bvals1, bvecs1, big_delta=Delta1, small_delta=delta1)
gtab2 = grad.gradient_table(bvals2, bvecs2, big_delta=Delta2, small_delta=delta2)

In [8]:
sphere = dpd.get_sphere()

In [9]:
#sphere = sphere.subdivide(1)

In [10]:
#sphere.x.shape

In [11]:
model1 = Model(gtab1, isotropic=BiExponentialIsotropicModel, sphere=sphere)#, alpha=0.000000001, l1_ratio=0.01)
model2 = Model(gtab2, isotropic=BiExponentialIsotropicModel, sphere=sphere)

In [12]:
fit1 = model1.fit(data1, te1)
fit2 = model2.fit(data2, te2)

In [13]:
predict12 = fit1.predict(gtab2, te2)
predict21 = fit2.predict(gtab1, te1)
predict11 = fit1.predict(gtab1, te1)
predict22 = fit2.predict(gtab2, te2)

In [14]:
fit1.beta.shape

(2172,)

In [15]:
print("Fit 1 => Predict 2: LSE=%s"%utils.LSE(predict12, data2))
print("Fit 2 => Predict 1: LSE=%s"%utils.LSE(predict21, data1))
print("Fit 1 => Predict 1: LSE=%s"%utils.LSE(predict11, data1))
print("Fit 2 => Predict 2: LSE=%s"%utils.LSE(predict22, data2))

Fit 1 => Predict 2: LSE=2.79178139128
Fit 2 => Predict 1: LSE=2.74690596909
Fit 1 => Predict 1: LSE=2.7284170543
Fit 2 => Predict 2: LSE=2.80331388191


In [16]:
np.mean([utils.LSE(predict12, data2), utils.LSE(predict21, data1)])

2.769343680184766